In [ ]:
# we want a read and write for the bulk data
# there will be different read and write functions for different data types

# in the end data will be read in based on the schema of the export file
    # but then put into the bulk data schema and with the normal schema

# writing 5 minute data files to the right location will be a part of it
# splitting the input data into the 5 minute segments will also be a part of it 


In [ ]:
# so for the recent captures what would I do
# I'll have a dictionary of camera names and metadata for the bulk data location

# I'll go through a sorted list of the files
# I'll grab the device name from the folder name
# I'll go into the folder and combine all the parquets and write < ez jk I need these for the quereies
# for the bulk data, read it one file at a time and query the data for each 5 minute segment of the day
# this can use the same query function that could be used for querying the bulk data

# it will take in a bounds of times and retrun the frames between those times
# then a different function will take those frames and write them

# it does involve some way of linking the data to the time associated with the data
# let's just start with the video data I have 

# for every 5 minute segment
# search the deduplicated file names for the relevant files
# read the parquet and line up the times with the frame counts

# the metadata should have the time, but ngl it could only have the time
# we could get the frame offset by querieng the times for the 5 minute segment
# and then lining up the indexes

# in that case we can just write all the files and hope they line up
# although taking it file by file might help keep errors from propigating
# if I had good tests I could trust this would be less of an issue
# acctually if the timestamps line up there's no issues
# this is only a problem for the old capture data beofre I fixed that timestamp bug
# I will still write it file by file to make it so I only have to write one 

# this schema does mean that the files themselves will be of gratly varying length due to the timestamps
# as little as 20 frames per video in timelapse mode which would result in low compression
# this is why we had the heterogenious timesamp structure anyway

# the essence then could be the same thing, but we treat the file names as the indcators for where the data is
# then we query the timestampsdf for that videos frames and calculate offsets for the desired times
# return the frames that are 

# so the takeaway from this is that we can just cut off by day (which will still take some code for audio)
# but for the camera structure we don't need to do anything since it's already divided that way.

# so back to the standard way I was doing it where I just transfer all the videos as they are to the new day folder
# I can also just add all the timestamps to the metadata raw and just make a check incase there are too many timestamps



In [ ]:
from datetime import datetime
import pandas as pd
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + "/dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import writeWorkingTSDf, readWorkingTSDF
recentCapPath = repoPath + "/recentCaptures/"

In [ ]:
# dictionaries
from collections import OrderedDict
fieldNames = ["responsiblePartyName", "instanceName", "developingPartyName", "deviceName", "dataType", "dataSource"]
myCams = {}
myCams["mobilepi"] = ["abhik", "mobilepi", "abhik", "piCam-raspberryPi5-Camv3120noir", "mp4", "piVidCap"]
myCams["bedroompi"] = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
myCams["bathroompi"] = ["abhik", "bathroompi", "abhik", "piCam-raspberryPi5-Camv3noir", "mp4", "piVidCap"]
myCams["bathroomCam"] = ["abhik", "bathroomCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
myCams["deskCam"] = ["abhik", "deskCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
# right now we are going through and writing the intake code for recent video captures
# later I would like to refactor the pividcap and the normal vidcap
# and clear out everything un needed for video capture in the repo
# I honestly should be writing this in data import


# damn I have a bunch of files seperated by the current timezone day and not the UTC day
# this does mean that there will be some cutting together and apart of the videos

In [ ]:
# read in the metadata parquet
workingdfs = {}
for k in myCams.keys():
    workingdfs[k] = readWorkingTSDF(*myCams[k])


foldersToImport = os.listdir(recentCapPath)
for f in foldersToImport:
    camName = f.split("_")[0]
    # get a list of file names
    fileNames = set([x.split(".")[0] for x in os.listdir(recentCapPath + f)])
    # combine all parquets
    for i, fileName in enumerate(fileNames):
        source = recentCapPath + f + "/" + fileName + ".parquet"
        if i == 0:
            readDf = pd.read_parquet(source)
        else:
            readDf = pd.concat(readDf, pd.read_parquet(source))

    # add to the full metadata parquet
    workingdfs[camName] = pd.concat(workingdfs[camName], readDf).sort_index()

    # for each file in a folder
        # if not a boundary file move and contuinue
        
        # the file contains more than one day of data
        # calc a list of days the file covers
        # read the video file into memory
        # for every day
            # define the new name
            # query the metadata parquet for the frame indexes for that day
            # write that slice of the video data to the new location
    
    # delete the folder

    pass

# we'll want a function that takes in a file name
# calculates all the days it covers
# iterates over each of the bounds for those days
    # queries the indexes from the data frame
    # saves those indexes to a new file

# honestly this can be folded into the normal moving processes